In [34]:
import json
import numpy as np
import math
import matplotlib.pyplot as plt

In [20]:
with open('out.json') as f:
  joint_data = json.load(f)

# Output:
#print(joint_data)

type(joint_data)

dict

In [21]:
#1) We want to check each dictionary within 'frames' data list until we get to a dictionary where 'num_bodies' isn't 0 
#Then, we put all dictionaries corresponding to detected body info in another list called Timeframes_detected

Timeframes_detected = []
timestep_data = joint_data['frames']
print(type(timestep_data)) #Confirm this is a list

for i in range(len(timestep_data)):
    if(timestep_data[i]['num_bodies'] ==1): #If there is a body detected at time corresponding to ith dict...
        Timeframes_detected.append(timestep_data[i]) #Append the dict with body info at that timestep to a new list
    
print(type(Timeframes_detected)) #Confirm this is a list
#print(Timeframes_detected)

#2) For each dictionary within the detected bodies list, we want to get ONLY the list of joint positions and the timestep
#Could've done this earlier, but may need other data later..

joint_pos_timesteps = []
for i in range(len(Timeframes_detected)):
    joint_pos_list = Timeframes_detected[i]['bodies'][0]['joint_positions'] #This is a list of lists (gives 3 coords for each joint for timestep corresponding to ith index)
    #Note: Since the 'bodies' key corresponds to a list of just 1 dict, the list only has 1 value so we use 0 index above
    timestep = Timeframes_detected[i]['timestamp_usec']
    temp_dict = {
                'time': timestep,
                'joint_positions': joint_pos_list
                }
    
    joint_pos_timesteps.append(temp_dict)
    
print(type(joint_pos_timesteps[1]))
print(joint_pos_timesteps)
#NOW, joint_pos_timesteps is in the format (for ex): [{'time': 183333, 'joint_positions': [[1,2,3],[-0.2,1,0.5],etc]}]
#Also, I think timesteps are measured in seconds from start

<class 'list'>
<class 'list'>
<class 'dict'>
[{'time': 1466888, 'joint_positions': [[-853.8265991210938, 168.40988159179688, 1203.6947021484375], [-852.507568359375, 11.890008926391602, 1182.53857421875], [-851.7507934570312, -114.1106185913086, 1174.29443359375], [-850.2789916992188, -305.25518798828125, 1194.457763671875], [-880.2991333007812, -274.3793640136719, 1189.113037109375], [-999.7002563476562, -233.48587036132812, 1198.5313720703125], [-1069.828125, -5.131006240844727, 1251.3743896484375], [-1079.7218017578125, 165.5253143310547, 1365.1444091796875], [-1065.4410400390625, 236.30686950683594, 1410.463623046875], [-1028.5008544921875, 320.5774230957031, 1436.646728515625], [-1008.9647827148438, 257.9344482421875, 1419.681884765625], [-820.7713623046875, -273.16015625, 1191.28857421875], [-713.52734375, -230.19776916503906, 1205.7479248046875], [-683.8612670898438, 16.649141311645508, 1223.88671875], [-662.886474609375, 222.5790252685547, 1243.6932373046875], [-662.00885009765

In [22]:
#3) We want to make a list of the form: [ [[1,2,3],[3,-2,9],[14,29,31],[2,1,0]], [[-0.2,5,3],[2,3,9],[-9,-6,8],[2,3,-14]], etc..]
#So each list within outer list correspond to a timestep, and each list (of 4 total) within inner lists corresponds to..
#..the hip, knee, ankle, and foot joints respectively

#To do this, we need to first get which indices correspond to the hip, knee, ankle, and foot joints. For this we look at the..
#.."joint_names" data 
#FIRST, we look at left side! 

hip_left = joint_data['joint_names'].index('HIP_LEFT')
knee_left = joint_data['joint_names'].index('KNEE_LEFT')
ankle_left = joint_data['joint_names'].index('ANKLE_LEFT')
foot_left = joint_data['joint_names'].index('FOOT_LEFT')

hip_right = joint_data['joint_names'].index('HIP_RIGHT')
knee_right = joint_data['joint_names'].index('KNEE_RIGHT')
ankle_right = joint_data['joint_names'].index('ANKLE_RIGHT')
foot_right = joint_data['joint_names'].index('FOOT_RIGHT')


left_leg_joints = []
right_leg_joints = []
for i in range(len(joint_pos_timesteps)):
    temp = [] #List of 4 joint pos lists, correspond to ith timestep in joint_pos_timesteps
    temp.append(joint_pos_timesteps[i]['joint_positions'][hip_left])
    temp.append(joint_pos_timesteps[i]['joint_positions'][knee_left])
    temp.append(joint_pos_timesteps[i]['joint_positions'][ankle_left])
    temp.append(joint_pos_timesteps[i]['joint_positions'][foot_left])
    left_leg_joints.append(temp)
    
    temp = [] #List of 4 joint pos lists, correspond to ith timestep in joint_pos_timesteps
    temp.append(joint_pos_timesteps[i]['joint_positions'][hip_right])
    temp.append(joint_pos_timesteps[i]['joint_positions'][knee_right])
    temp.append(joint_pos_timesteps[i]['joint_positions'][ankle_right])
    temp.append(joint_pos_timesteps[i]['joint_positions'][foot_right])
    right_leg_joints.append(temp)
    
#print(left_leg_joints)



In [23]:
#4) #Calculate angle between hip/knee joints for each leg 

    # a) We start with the hip/knee angle as drawn in my notes.
        #First, we need to find knee-hip and ankle-knee differences

left_hk_differences = []
for i in range(len(left_leg_joints)):
    diff_x = left_leg_joints[i][1][0] - left_leg_joints[i][0][0]
    diff_y = left_leg_joints[i][1][1] - left_leg_joints[i][0][1]
    diff_z = left_leg_joints[i][1][2] - left_leg_joints[i][0][2]
    diff_hk = [diff_x, diff_y, diff_z]
    
    diff_x2 = left_leg_joints[i][2][0] - left_leg_joints[i][1][0]
    diff_y2 = left_leg_joints[i][2][1] - left_leg_joints[i][1][1]
    diff_z2 = left_leg_joints[i][2][2] - left_leg_joints[i][1][2]
    diff_ak = [diff_x2, diff_y2,diff_z2]
    
    diff = [diff_hk, diff_ak]
    left_hk_differences.append(diff)
 

#Now, left_hk differences is in the form (for ex): [ [[1,2,3],[4,9,10]], [[8,9,10],[-2,3,6]], etc..] where..
#.. each inner list correspond to a timestep, the first list inside each timestep list is the knee-hip, and second list is..
#..ankle-knee
left_hk_diffs = np.asarray(left_hk_differences)
print(type(left_hk_diffs))
print(left_hk_diffs) #To make sure it works!



<class 'numpy.ndarray'>
[[[ -37.24981689  284.65559387  207.85705566]
  [  16.62097168  327.24633789  -85.84228516]]

 [[ -53.80511475  307.0664978   171.99084473]
  [   4.1315918   331.75131226  -75.53063965]]

 [[ -56.3034668   317.7277832   156.53649902]
  [  -2.66992188  333.34313965  -79.71240234]]

 ...

 [[  -3.59625244  409.10421753  109.41394043]
  [-107.3918457   364.918396    138.24084473]]

 [[   1.9864502   413.7253418   132.35559082]
  [ -72.88330078  368.0055542   177.76330566]]

 [[  -3.90863037  352.92709351  136.09191895]
  [ -43.21295166  315.64105225  170.87280273]]]


In [24]:
#4) Continued..
#Now, we want to find the hip-knee angle, so angle between the two vectors in each timestep in left_hk_diffs
#. We can use Numpy's dot product function: np.dot(a,b), AND..
#..Numpy's vector length calculation function: np.linalg.norm(x), AND inverse cosine: math.acos(a)

left_leg_hk_angles = [] #This will be a list of angles between the hip and the knee for each timestep! This is in degrees

for i in range(len(left_hk_diffs)):
    length_hk = np.linalg.norm(left_hk_diffs[i][0]) #Magnitude of knee-hip vector 
    length_ak = np.linalg.norm(left_hk_diffs[i][1]) #Magnitude of ankle-knee vector
    dot_prod = np.dot(left_hk_diffs[i][0], left_hk_diffs[i][1]) #Dot product of the two vectors
    angle = math.acos(dot_prod/(length_hk*length_ak)) #Angle between the two vectors
    
    angle = angle * (180/(np.pi))
    left_leg_hk_angles.append(angle) 



print(left_leg_hk_angles) 


[51.52769476333242, 42.96241420551419, 40.419904370148174, 41.003409858878975, 33.60773083386051, 26.805807153768043, 19.66446806950134, 13.955047762544476, 16.890360954760663, 20.500957061816234, 21.054655112778573, 26.323302337313486, 29.806294710674027, 29.601795290671863, 32.56119523322973, 30.674017964049185, 26.494963126040364, 24.273393738898744, 24.207208015360578, 27.961248041790913, 31.09332368766793, 32.692823468393385, 34.15937028308055, 30.20846242876136, 29.94806091879814, 27.27473976926454, 29.16984242754594, 27.83497907724333, 23.288428532530656, 21.94575998046618, 22.641606600419102, 25.52817653394827, 22.64399789681528, 23.165474593586744, 21.564224956586173, 25.778994960184995, 28.03081619031879, 26.70159185044741, 42.08627414799842, 27.2292529154686, 40.469909219102654, 42.21210688092706, 38.51426878643138, 38.31500922283899, 41.11393313499523, 39.44904860676685, 36.017518600986634, 28.170131960507554, 17.562424368028616, 6.114676319718732, 4.7171982725036745, 6.485

In [25]:
#4) Continued..
    #b) Now we want to calculate right leg hip-knee angles
    #First, find knee-hip and ankle-knee as we did for left leg
    
right_hk_differences = []
for i in range(len(right_leg_joints)):
    diff_x = right_leg_joints[i][1][0] - right_leg_joints[i][0][0]
    diff_y = right_leg_joints[i][1][1] - right_leg_joints[i][0][1]
    diff_z = right_leg_joints[i][1][2] - right_leg_joints[i][0][2]
    diff_hk = [diff_x, diff_y, diff_z]
    
    diff_x2 = right_leg_joints[i][2][0] - right_leg_joints[i][1][0]
    diff_y2 = right_leg_joints[i][2][1] - right_leg_joints[i][1][1]
    diff_z2 = right_leg_joints[i][2][2] - right_leg_joints[i][1][2]
    diff_ak = [diff_x2, diff_y2,diff_z2]
    
    diff = [diff_hk, diff_ak]
    right_hk_differences.append(diff)
 

#Now, right_hk differences is in the form (for ex): [ [[1,2,3],[4,9,10]], [[8,9,10],[-2,3,6]], etc..] where..
#.. each inner list correspond to a timestep, the first list inside each timestep list is the knee-hip, and second list is..
#..ankle-knee
right_hk_diffs = np.asarray(right_hk_differences)
print(type(right_hk_diffs))
print(right_hk_diffs) #To make sure it works!


<class 'numpy.ndarray'>
[[[ 56.99682617 271.70970154 219.71765137]
  [ -5.84655762 334.60644531 -71.87744141]]

 [[ 32.81890869 305.40356445 179.28881836]
  [-13.98150635 338.30587769 -59.85974121]]

 [[ 20.30023193 315.18226624 169.10351562]
  [-15.02459717 339.56512451 -66.58239746]]

 ...

 [[ 57.49871826 419.02642822   8.68908691]
  [-11.18652344 400.53051758  81.99841309]]

 [[ 50.48022461 429.56382751  34.80480957]
  [ 20.33282471 409.26904297  89.83435059]]

 [[-20.38989258 363.71592712 100.40405273]
  [-35.05969238 331.82012939 148.74316406]]]


In [26]:
#4) continued...
#Now, we want to find the hip-knee angle, so angle between the two vectors in each timestep in right_hk_diffs
#. We can use Numpy's dot product function: np.dot(a,b), AND..
#..Numpy's vector length calculation function: np.linalg.norm(x), AND inverse cosine: math.acos(a)

right_leg_hk_angles = [] #This will be a list of angles between the hip and the knee for each timestep! This is in degrees

for i in range(len(right_hk_diffs)):
    length_hk = np.linalg.norm(right_hk_diffs[i][0]) #Magnitude of knee-hip vector 
    length_ak = np.linalg.norm(right_hk_diffs[i][1]) #Magnitude of ankle-knee vector
    dot_prod = np.dot(right_hk_diffs[i][0], left_hk_diffs[i][1]) #Dot product of the two vectors
    angle = math.acos(dot_prod/(length_hk*length_ak)) #Angle between the two vectors
    
    angle = angle * (180/(np.pi))
    right_leg_hk_angles.append(angle) 



print(right_leg_hk_angles) 


[54.13415793709661, 44.03861508127548, 42.46922264584975, 42.31894471364941, 35.53219647815807, 27.751559725750646, 22.696063880353627, 20.773392895538983, 25.832704830677983, 31.771535491340735, 33.17744513813804, 37.105863069828246, 37.88833000688583, 36.21810849383356, 37.85903608915672, 34.90998130308442, 31.221447426545094, 21.722871120443912, 21.649529472426106, 25.272887686174784, 30.53756597025536, 33.0057509477234, 36.36522576835018, 36.72733286523411, 37.618052221443435, 37.267646745513424, 39.50424719539047, 39.24190176536029, 37.32837890879413, 37.087391624718414, 39.20335882939837, 41.09817515296514, 40.90194334807343, 41.74336831613354, 41.6728077726232, 44.64092328444231, 45.42629997959893, 42.27565091131205, 44.989626166243774, 36.519779122389075, 38.707868184488724, 35.46427001538825, 28.689149826395656, 26.301392414469202, 25.638462707169445, 22.675584129836366, 18.283486851152293, 14.88042889992577, 12.49947689199464, 11.237712418319212, 13.210285748873442, 13.652000

In [27]:
#5) Calculate angle between foot and ankle for each leg 

 

SyntaxError: invalid syntax (<ipython-input-27-e103ffe5fde5>, line 3)

In [ ]:
#NOW start the plotting process
#My thought process is to 1) Get a list of each pair of joint indices that need to be connected. Right now they're given as names...
#so the first for loop below does that
#2) Plot 1 plot with first, scatterpoints of x,y coordinates on it. Then, connect the desired point pairs..


joint_list = joint_data['joint_names']
joint_combos = []
joint_combo_names = joint_data['bone_list']

for joint_combo in joint_combo_names: #For each joint combination/pairing..
    index1 = joint_list.index(joint_combo[0]) #Numerical index of first joint in this pair from the joint_names list (order of actual given joint coords)
    index2 = joint_list.index(joint_combo[1]) #Numerical index of second joint...etc
    numerical_indices_thispair = [index1, index2]
    joint_combos.append(numerical_indices_thispair)
    
print(joint_combos)
joint_combos = np.array(joint_combos)


#for i in range(number_of_postures):
i = 0; #Temporary; just to run it once
while(i!=1):
    fig, ax = plt.subplots(1, figsize=(3, 8))
    plt.title('Skeleton')
    plt.xlim(100, 180)
    plt.ylim(-250, 0)
    #skeleton = movement[i*20:(i+1)*20]
    #MY EDIT: Get x, y,z positions of each joint for FIRST timestep only at first
    for joint in joint_pos_timesteps[0]['joint_positions']:
        x = joint[0]
        y = joint[1]
        z = joint[2]
        
    sc = ax.scatter(x, y)
    #for bone in bone_list:
     #   ax.plot([x[bone[0]], x[bone[1]]], [y[bone[0]], y[bone[1]]], 'r')


    for joint_combo in joint_combos:
        #We are using joint_pos_timesteps array that was previously derived from the joint_data..
        #..joint_pos_timesteps gives 3D joint positions for every timestep along with the timestep, so: [{'timestep':999, 'joint_position': [[1,2,3],[4,5,6],etc..]}, {'timestep':etc}]
        joint_positions_firsttime = np.array(joint_pos_timesteps[0]['joint_positions']) #List of lists
        x1 = joint_positions_firsttime[joint_combo[0]][0] #X and y coordinates of first joint in pair
        y1 = joint_positions_firsttime[joint_combo[0]][1]

        x2 = joint_positions_firsttime[joint_combo[1]][0] #x and y coords of second joint in pair
        y2 = joint_positions_firsttime[joint_combo[1]][1]

        plt.plot([x1,y1],[x2,y2],'r')
#plt.show()
    